# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 11 2022 8:47AM,241494,19,TR-RG-07142022_WM5,"GUSA Granules USA, Inc.",Released
1,Jul 11 2022 8:30AM,241460,10,PRF-32969,Bio-PRF,Released
2,Jul 11 2022 8:30AM,241460,10,PRF-32977,Bio-PRF,Released
3,Jul 11 2022 8:30AM,241460,10,PRF-32981,Bio-PRF,Released
4,Jul 11 2022 8:30AM,241460,10,PRF-32984,Bio-PRF,Released
5,Jul 11 2022 8:30AM,241460,10,PRF-32987,Bio-PRF,Released
6,Jul 11 2022 8:30AM,241460,10,PRF-32994,Bio-PRF,Released
7,Jul 11 2022 8:30AM,241460,10,PRF-33000,Bio-PRF,Released
8,Jul 11 2022 8:30AM,241457,10,8249216,Eywa Pharma Inc.,Released
9,Jul 11 2022 8:30AM,241463,10,Enova-10266,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,241481,Released,1
17,241482,Released,13
18,241483,Released,1
19,241484,Released,6
20,241494,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241481,NaN,1.0
241482,NaN,13.0
241483,NaN,1.0
241484,NaN,6.0
241494,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241457,0.0,1.0
241460,0.0,7.0
241463,0.0,2.0
241468,0.0,1.0
241471,1.0,27.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241457,0,1
241460,0,7
241463,0,2
241468,0,1
241471,1,27


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241457,0,1
1,241460,0,7
2,241463,0,2
3,241468,0,1
4,241471,1,27


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241457,,1
1,241460,,7
2,241463,,2
3,241468,,1
4,241471,1,27


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 11 2022 8:47AM,241494,19,"GUSA Granules USA, Inc."
1,Jul 11 2022 8:30AM,241460,10,Bio-PRF
8,Jul 11 2022 8:30AM,241457,10,Eywa Pharma Inc.
9,Jul 11 2022 8:30AM,241463,10,Emerginnova
11,Jul 11 2022 8:30AM,241468,10,Livs Products
12,Jul 11 2022 8:25AM,241483,18,"Emersa Waterbox, LLC"
13,Jul 11 2022 8:25AM,241484,10,"Amcyte Pharma, Inc."
19,Jul 11 2022 8:24AM,241482,10,Beach Products Inc
32,Jul 11 2022 8:22AM,241480,16,"Emersa Waterbox, LLC"
33,Jul 11 2022 8:22AM,241481,16,Sartorius Stedim North America


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 11 2022 8:47AM,241494,19,"GUSA Granules USA, Inc.",,1
1,Jul 11 2022 8:30AM,241460,10,Bio-PRF,,7
2,Jul 11 2022 8:30AM,241457,10,Eywa Pharma Inc.,,1
3,Jul 11 2022 8:30AM,241463,10,Emerginnova,,2
4,Jul 11 2022 8:30AM,241468,10,Livs Products,,1
5,Jul 11 2022 8:25AM,241483,18,"Emersa Waterbox, LLC",,1
6,Jul 11 2022 8:25AM,241484,10,"Amcyte Pharma, Inc.",,6
7,Jul 11 2022 8:24AM,241482,10,Beach Products Inc,,13
8,Jul 11 2022 8:22AM,241480,16,"Emersa Waterbox, LLC",,1
9,Jul 11 2022 8:22AM,241481,16,Sartorius Stedim North America,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 11 2022 8:47AM,241494,19,"GUSA Granules USA, Inc.",1,
1,Jul 11 2022 8:30AM,241460,10,Bio-PRF,7,
2,Jul 11 2022 8:30AM,241457,10,Eywa Pharma Inc.,1,
3,Jul 11 2022 8:30AM,241463,10,Emerginnova,2,
4,Jul 11 2022 8:30AM,241468,10,Livs Products,1,
5,Jul 11 2022 8:25AM,241483,18,"Emersa Waterbox, LLC",1,
6,Jul 11 2022 8:25AM,241484,10,"Amcyte Pharma, Inc.",6,
7,Jul 11 2022 8:24AM,241482,10,Beach Products Inc,13,
8,Jul 11 2022 8:22AM,241480,16,"Emersa Waterbox, LLC",1,
9,Jul 11 2022 8:22AM,241481,16,Sartorius Stedim North America,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 11 2022 8:47AM,241494,19,"GUSA Granules USA, Inc.",1,
1,Jul 11 2022 8:30AM,241460,10,Bio-PRF,7,
2,Jul 11 2022 8:30AM,241457,10,Eywa Pharma Inc.,1,
3,Jul 11 2022 8:30AM,241463,10,Emerginnova,2,
4,Jul 11 2022 8:30AM,241468,10,Livs Products,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 11 2022 8:47AM,241494,19,"GUSA Granules USA, Inc.",1,NaN
1,Jul 11 2022 8:30AM,241460,10,Bio-PRF,7,NaN
2,Jul 11 2022 8:30AM,241457,10,Eywa Pharma Inc.,1,NaN
3,Jul 11 2022 8:30AM,241463,10,Emerginnova,2,NaN
4,Jul 11 2022 8:30AM,241468,10,Livs Products,1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 11 2022 8:47AM,241494,19,"GUSA Granules USA, Inc.",1,0.0
1,Jul 11 2022 8:30AM,241460,10,Bio-PRF,7,0.0
2,Jul 11 2022 8:30AM,241457,10,Eywa Pharma Inc.,1,0.0
3,Jul 11 2022 8:30AM,241463,10,Emerginnova,2,0.0
4,Jul 11 2022 8:30AM,241468,10,Livs Products,1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2173245,50,0.0
16,482961,2,0.0
18,241483,1,0.0
19,1690338,91,4.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2173245,50,0.0
1,16,482961,2,0.0
2,18,241483,1,0.0
3,19,1690338,91,4.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,50,0.0
1,16,2,0.0
2,18,1,0.0
3,19,91,4.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,50.0
1,16,Released,2.0
2,18,Released,1.0
3,19,Released,91.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19
Status,,,,
Executing,0.0,0.0,0.0,4.0
Released,50.0,2.0,1.0,91.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19
0,Executing,0.0,0.0,0.0,4.0
1,Released,50.0,2.0,1.0,91.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19
0,Executing,0.0,0.0,0.0,4.0
1,Released,50.0,2.0,1.0,91.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()